In [1]:
import string
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
import joblib
import pandas as pd

In [2]:
true=pd.read_csv("True.csv")
fake=pd.read_csv("Fake.csv")
true['label']=1
fake['label']=0


In [3]:
data=pd.concat((fake,true),axis=0)
data.head(1)

,title,text,subject,date,label
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0


In [4]:
data= data.drop(['title', 'subject', 'date'], axis=1)

In [5]:
data.reset_index(inplace=True)

In [6]:
data.drop(['index'],axis=1,inplace=True)

In [7]:
def preprocess_text(text):
    text = text.lower()  # lowercase
    text = re.sub(r"http\S+|www\S+|https\S+", '', text, flags=re.MULTILINE)  # remove URLs
    text = re.sub(r'\@w+|\#', '', text)  # remove mentions and hashtags
    text = re.sub(f"[{re.escape(string.punctuation)}]", '', text)  # remove punctuation
    text = re.sub(r'\d+', '', text)  # remove digits
    text = text.strip()
    return text

data['text'] = data['text'].astype(str).apply(preprocess_text)


In [8]:
data.head()

,text,label
0,donald trump just couldn t wish all americans ...,0
1,house intelligence committee chairman devin nu...,0
2,on friday it was revealed that former milwauke...,0
3,on christmas day donald trump announced that h...,0
4,pope francis used his annual christmas day mes...,0


In [13]:
X_train, X_test, y_train, y_test = train_test_split(data['text'], data['label'], test_size=0.2, random_state=42)

# === Step 5: TF-IDF Vectorization ===
vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# === Step 6: Train Logistic Regression ===
model = LogisticRegression(max_iter=1000)
model.fit(X_train_tfidf, y_train)

# === Step 7: Evaluate ===
y_pred = model.predict(X_test_tfidf)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))
# === Step 8: Save model and vectorizer ===
joblib.dump(vectorizer, 'vectorizer.jb')
joblib.dump(model, 'lr_model.jb')

Accuracy: 0.9866369710467706
              precision    recall  f1-score   support

           0       0.99      0.99      0.99      4733
           1       0.98      0.99      0.99      4247

    accuracy                           0.99      8980
   macro avg       0.99      0.99      0.99      8980
weighted avg       0.99      0.99      0.99      8980



['lr_model.jb']